In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-127964")
exp = Experiment(workspace=ws, name="quick-starts-ws-127964")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-127559
Azure region: southcentralus
Subscription id: 54424c07-8d8d-4896-9c92-b731a63208d3
Resource group: aml-quickstarts-127559


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# choose a name for your cluster
cluster_name= "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws , name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target ...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2' , max_nodes=4)


#create the cluster
compute_target=ComputeTarget.create(ws,cluster_name,
compute_config)

Creating a new compute target ...


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps =RandomParameterSampling({
    "--C": uniform(0.6,1),
    "--max_iter": uniform(100,110)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est =ScriptRunConfig(source_directory='training'
                     script='train.py'
                     arguments=['--C',1.0,'--max_iter',100],
                     compute_target=compute_target
                     )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =HyperDriveConfig(estimator=est
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE(),
                                    max_total_runs=20,
                                    max_concurrent_runs=4 
                                    
                                    )

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run=exp.submit(hyperdrive_config,show_output=True)
RunDetails(hyperdrive_run).show()

In [ ]:
#To periodically check the status of the run object, and navigate to Azure portal to monitor the run
hyperdrive_run

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())

In [ ]:
#save the best model
model =best_run.register_model(model_name = 'best_model', model_path='model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
web_path ='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'


In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###